In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os
import json
import time
import concurrent.futures
import pandas as pd


In [ ]:
def get_invoice_info(cookie,doc_id):
    url = f"https://ecosys.gov.vn/CertificatesUpgrade/Business/OrderDisplay.aspx?DocId={doc_id}"

    # Headers cần thiết
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": cookie,
        "DNT": "1",
        "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        print("Request thành công!")
        # print(response.text)  # In nội dung HTML của trang
    else:
        print(f"Lỗi: {response.status_code}")
        # print(response.text)

    invoice_info = {}
    invoice_info["doc_id"] = doc_id

    if response.text:
        soup = BeautifulSoup(response.text,"html.parser")
    info_soup = soup.find('div',id = "ctl00_cplhContainer_rpvRepresenter") 
    audit_info_soup = soup.find('div',id = "ctl00_cplhContainer_pnlAuditTrail") 


    try :
        tt = info_soup.find('span',id = "ctl00_cplhContainer_lblStatus").get_text().strip()
        invoice_info["status"] = tt
    except:
        print("not found status")
    try :
        ordercode = info_soup.find('span',id = "ctl00_cplhContainer_lblOrderCode").get_text().strip()
        invoice_info["ordercode"] = ordercode
    except:
        print("not found ordercode")
    try :
        companyname = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyName").get_text().strip()
        invoice_info["companyname"] = companyname
    except:
        print("not found companyname")
    try :
        companytaxcode = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyTaxcode").get_text().strip()
        invoice_info["companytaxcode"] = companytaxcode
    except:
        print("not found companytaxcode")
    try :
        companyaddress = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyAddress").get_text().strip()
        invoice_info["companyaddress"] = companyaddress
    except:
        print("not found companyaddress")
    try :
        companyemail = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyEmail").get_text().strip()
        invoice_info["companyemail"] = companyemail
    except:
        print("not found companyemail")
    try :
        amount = info_soup.find('span',id = "ctl00_cplhContainer_lblAmount").get_text().strip()
        invoice_info["amount"] = amount
    except:
        print("not found amount")
        
    try:
        co_table = info_soup.find('table',class_ = 'rgMasterTable')
        # Lấy tiêu đề
        co_table_header = [th.get_text(separator=" ", strip=True) for th in co_table.find('thead').find_all('th', class_ = "rgHeader")]

        # Lấy dữ liệu từng dòng trong bảng
        co_info_rows = []
        for tr in co_table.find('tbody').find_all('tr'):
            row = [td.get_text(separator=" ", strip=True) for td in tr.find_all('td')]
            co_info_rows.append(row)
    except:
        print("Not found goods_info")
    co_info = [co_table_header] + co_info_rows
    invoice_info["co_info"] = co_info
    
    try :
        invoice_code = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceCode").get_text().strip()
        invoice_info["invoice_code"] = invoice_code
    except:
        print("not found amoinvoice_codeunt")
    try :
        invoice_receip_no = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceReceiptNumber").get_text().strip()
        invoice_info["invoice_receip_no"] = invoice_receip_no
    except:
        print("not found invoice_receip_no")
    try :
        invoice_address_label = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceAddressLabel")
        
        if invoice_address_label:
            try :
                a_element = invoice_address_label.find_next('a')  
                invoice_address_label = a_element['href'].strip()  
                # print(invoice_address_label)
            except:
                print("Thẻ <a> không tìm thấy hoặc không có thuộc tính href.") 
            
        invoice_info["invoice_address_label"] = invoice_address_label
    except:
        print("not found invoice_address_label")
    try :
        another_email = info_soup.find('span',id = "ctl00_cplhContainer_lblEmail01").get_text().strip()
        invoice_info["another_email"] = another_email
    except:
        print("not found another_email")    
        
    try :
        created_at = audit_info_soup.find('span',id = "ctl00_cplhContainer_AuditTrail1_lblCreatedAt").get_text().strip()
        invoice_info["created_at"] = created_at
    except:
        print("not found created_at")   
        
    try :
        last_modified_at = audit_info_soup.find('span',id = "ctl00_cplhContainer_AuditTrail1_lblLastModifiedAt").get_text().strip()
        invoice_info["last_modified_at"] = last_modified_at
    except:
        print("not found last_modified_at")   

    print(invoice_info)
    return invoice_info

In [ ]:

cookie = 'Bifrost_SessionID=q0svamuliir35pmr0xag1c5t'

In [ ]:

# Hàm lưu trạng thái
def save_state(states, thread_index):
    file_path = state_file_template.format(thread_index)
    with open(file_path, "w") as f:
        json.dump(states, f, ensure_ascii=False, indent=4)  # Định dạng dễ đọc và tránh lỗi ghi

# Hàm đọc trạng thái
def load_state(thread_index):
    try:
        file_path = state_file_template.format(thread_index)
        if os.path.exists(file_path) and os.path.getsize(file_path) > 0:  # Kiểm tra file tồn tại và không trống
            with open(file_path, "r") as f:
                states = json.load(f)
                if isinstance(states, dict) and "start_index" in states:
                    return states
        # Nếu không hợp lệ hoặc không có 'start_index'
        return {"start_index": 0}
    except FileNotFoundError:
        return {"start_index": 0}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return {"start_index": 0}  # Khởi tạo lại nếu lỗi


# Hàm xử lý từng chunk dữ liệu
def process_data_chunk(data_chunk, thread_index, start_index):
    all_invoice_info = [] 
    error_stt_list = []
    current_index = start_index
    MAX_RETRIES = 10
    RETRY_DELAY = 3  # Giây

    while True:
        try:
            for index, item in enumerate(data_chunk[start_index:], start=start_index):
                retries = 0
                success = False

                while retries < MAX_RETRIES and not success:
                    try:
                        invoice_info = get_invoice_info(cookie, item)
                        all_invoice_info.append(invoice_info)
                        success = True  # Xử lý thành công, thoát vòng lặp retry
                        
                        # Ghi kết quả vào file riêng cho thread
                        with open(output_file_template.format(thread_index), "a", encoding="utf-8") as f:
                            f.write(json.dumps(invoice_info, ensure_ascii=False) + "\n")
                    
                    except ConnectionError as e:
                        retries += 1
                        print(f"Connection error for invoice {item}. Retry {retries}/{MAX_RETRIES}. Error: {e}")
                        time.sleep(RETRY_DELAY)
                    
                    except Exception as e:
                        retries += 1
                        print(f"Error processing invoice {item}. Retry {retries}/{MAX_RETRIES}. Error: {e}")
                        time.sleep(RETRY_DELAY)

                if not success:
                    print(f"Failed to process invoice {item} after {MAX_RETRIES} retries.")
                    error_stt_list.append({"stt": item, "error": "Max retries exceeded"})
                    
                    # Ghi lỗi vào file
                    with open(error_file_template.format(thread_index), "a", encoding="utf-8") as f:
                        f.write(json.dumps({"stt": item, "error": "Max retries exceeded"}, ensure_ascii=False) + "\n")

                # Cập nhật trạng thái
                current_index = index + 1
                save_state({"start_index": current_index}, thread_index)

            # Hoàn thành chunk, trả về kết quả
            return all_invoice_info, current_index, error_stt_list

        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}. Restarting...")
            time.sleep(5)  # Đợi 5 giây trước khi thử lại

            
# Thư mục để lưu trạng thái và kết quả
output_dir = "org"
os.makedirs(output_dir, exist_ok=True)

state_file_template = os.path.join(output_dir, "loop_state_invoice_{}.json")
output_file_template = os.path.join(output_dir, "output_invoice_data_{}.txt")
error_file_template = os.path.join(output_dir, "error_invoice_ids_{}.txt")

# Chia danh sách thành các chunk
with open("output_invoice_ids.txt", "r", encoding="utf-8") as file:
    invoice_list = [line.strip() for line in file]

chunk_count = 10
chunk_size = -(-len(invoice_list) // chunk_count)
chunks = [invoice_list[i:i + chunk_size] for i in range(0, len(invoice_list), chunk_size)]

# Tải trạng thái đã lưu
saved_states = []
for i in range(len(chunks)):
    state = load_state(i)
    if "start_index" not in state:  # Đảm bảo có key 'start_index'
        state = {"start_index": 0}
    saved_states.append(state)

# Xử lý đa luồng
states = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=len(chunks)) as executor:
    futures = {
        executor.submit(process_data_chunk, chunk, idx, saved_states[idx]["start_index"]): idx
        for idx, chunk in enumerate(chunks)
    }
    for future in concurrent.futures.as_completed(futures):
        thread_index = futures[future]
        try:
            results, current_index, error_stts = future.result()
            states[thread_index] = {"start_index": current_index}
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}")
            save_state({"start_index": 0}, thread_index)

# Hợp nhất và lưu dữ liệu lỗi
error_file_names = [error_file_template.format(i) for i in range(chunk_count)]
all_error_data = []
for file_name in error_file_names:
    if os.path.exists(file_name):
        with open(file_name, "r", encoding="utf-8") as file:
            all_error_data.extend(json.loads(line) for line in file)

error_df = pd.DataFrame(all_error_data)
error_df.to_csv("all_error_invoice_id.csv", index=False, encoding="utf-8")

# Hợp nhất và lưu dữ liệu kết quả
output_file_names = [output_file_template.format(i) for i in range(chunk_count)]
all_output_data = []
for file_name in output_file_names:
    if os.path.exists(file_name):
        with open(file_name, "r", encoding="utf-8") as file:
            all_output_data.extend(json.loads(line) for line in file)

output_df = pd.DataFrame(all_output_data)
output_df.to_csv("all_invoice_info.csv", index=False, encoding="utf-8")


Request thành công!
{'doc_id': '944288', 'status': 'Đã nộp phí', 'ordercode': '0000E28B3', 'companyname': 'CÔNG TY TNHH ACE BONGSAN VINA', 'companytaxcode': '1102022318', 'companyaddress': 'O2-10 (thuộc nhà xưởng O2), Đường 1B, Khu xưởng Kizuna 3 mở rộng, Khu K1, KCN Tân Kim mở rộng, Thị trấn Cần Giuộc, Huyện Cần Giuộc, Tỉnh Long An, Việt Nam', 'companyemail': 'huynhthuy8509@gmail.com', 'amount': '60,000 (VNĐ)', 'co_info': [['Số C/O', 'Loại phí', 'Loại phí', 'Số tiền (VNĐ)'], ['VN-KR 24/02/029799', '1', 'C/O mới', '60,000']], 'invoice_code': '922845', 'invoice_receip_no': '1102022318-0000E28B3', 'invoice_address_label': 'https://0106196017.vnpt-invoice.com.vn', 'another_email': 'inv@eximuni.com', 'created_at': '27/09/2024 08:26:04 AM', 'last_modified_at': '28/09/2024 11:46:23 AM'}
Request thành công!
{'doc_id': '919591', 'status': 'Đã nộp phí', 'ordercode': '0000DC8FC', 'companyname': 'CÔNG TY TNHH ACE BONGSAN VINA', 'companytaxcode': '1102022318', 'companyaddress': 'O2-10 (thuộc nhà x